In [2]:
%load_ext autoreload
%autoreload 2
from typing import *
from __future__ import annotations

In [3]:
from utils import load_input

# Part 1

Your raft makes it to the tropical island; it turns out that the small crab was an excellent navigator. You make your way to the resort.

As you enter the lobby, you discover a small problem: the floor is being renovated. You can't even reach the check-in desk until they've finished installing the new tile floor.

The tiles are all hexagonal; they need to be arranged in a hex grid with a very specific color pattern. Not in the mood to wait, you offer to help figure out the pattern.

The tiles are all white on one side and black on the other. They start with the white side facing up. The lobby is large enough to fit whatever pattern might need to appear there.

A member of the renovation crew gives you a list of the tiles that need to be flipped over (your puzzle input). Each line in the list identifies a single tile that needs to be flipped by giving a series of steps starting from a reference tile in the very center of the room. (Every line starts from the same reference tile.)

Because the tiles are hexagonal, every tile has six neighbors: east, southeast, southwest, west, northwest, and northeast. These directions are given in your list, respectively, as e, se, sw, w, nw, and ne. A tile is identified by a series of these directions with no delimiters; for example, esenee identifies the tile you land on if you start at the reference tile and then move one tile east, one tile southeast, one tile northeast, and one tile east.

Each time a tile is identified, it flips from white to black or from black to white. Tiles might be flipped more than once. For example, a line like esew flips a tile immediately adjacent to the reference tile, and a line like nwwswee flips the reference tile itself.

Here is a larger example:
```
sesenwnenenewseeswwswswwnenewsewsw
neeenesenwnwwswnenewnwwsewnenwseswesw
seswneswswsenwwnwse
nwnwneseeswswnenewneswwnewseswneseene
swweswneswnenwsewnwneneseenw
eesenwseswswnenwswnwnwsewwnwsene
sewnenenenesenwsewnenwwwse
wenwwweseeeweswwwnwwe
wsweesenenewnwwnwsenewsenwwsesesenwne
neeswseenwwswnwswswnw
nenwswwsewswnenenewsenwsenwnesesenew
enewnwewneswsewnwswenweswnenwsenwsw
sweneswneswneneenwnewenewwneswswnese
swwesenesewenwneswnwwneseswwne
enesenwswwswneneswsenwnewswseenwsese
wnwnesenesenenwwnenwsewesewsesesew
nenewswnwewswnenesenwnesewesw
eneswnwswnwsenenwnwnwwseeswneewsenese
neswnwewnwnwseenwseesewsenwsweewe
wseweeenwnesenwwwswnew
```
In the above example, 10 tiles are flipped once (to black), and 5 more are flipped twice (to black, then back to white). After all of these instructions have been followed, a total of 10 tiles are black.

Go through the renovation crew's list and determine which tiles they need to flip. After all of the instructions have been followed, how many tiles are left with the black side up?




In [4]:
raw = load_input(24)

I think using decimals and having hexes identified by their center points works, but there must be an easier way

In [71]:
from collections import defaultdict, namedtuple

from decimal import *
getcontext().prec = 6


In [117]:
fractional_move = Decimal('0.8660245038')

In [118]:
move = {
    'e': lambda p: Point(p.x+1, p.y),
    'se': lambda p: Point(p.x+Decimal('0.5'), p.y-fractional_move),
    'sw': lambda p: Point(p.x-Decimal('0.5'), p.y-fractional_move),
    'w': lambda p: Point(p.x-1, p.y),
    'nw': lambda p: Point(p.x-Decimal('0.5'), p.y+fractional_move),
    'ne': lambda p: Point(p.x+Decimal('0.5'), p.y+fractional_move),
}

Point = namedtuple("Point", "x y") # hexagons are identifies by their central points

In [119]:
def locate_tile(instructions: str) -> Point:
    loc = Point(Decimal(0), Decimal(0))
    it = iter(instructions)
    while True:
        try:
            direction = next(it)
            if direction in ("n", "s"):
                direction += next(it)
            loc = move[direction](loc)
        except StopIteration:
            break
    return loc

In [120]:
tiles = defaultdict(lambda: 1) # use 1 for white, -1 for black
for instructions in raw:
    tile = locate_tile(instructions)
    tiles[tile] *= -1

In [121]:
len(list(filter(lambda x: x == -1, tiles.values())))

420

# Part 2

The tile floor in the lobby is meant to be a living art exhibit. Every day, the tiles are all flipped according to the following rules:

Any black tile with zero or more than 2 black tiles immediately adjacent to it is flipped to white.
Any white tile with exactly 2 black tiles immediately adjacent to it is flipped to black.
Here, tiles immediately adjacent means the six tiles directly touching the tile in question.

The rules are applied simultaneously to every tile; put another way, it is first determined which tiles need to be flipped, then they are all flipped at the same time.

In the above example, the number of black tiles that are facing up after the given number of days has passed is as follows:
```
Day 1: 15
Day 2: 12
Day 3: 25
Day 4: 14
Day 5: 23
Day 6: 28
Day 7: 41
Day 8: 37
Day 9: 49
Day 10: 37

Day 20: 132
Day 30: 259
Day 40: 406
Day 50: 566
Day 60: 788
Day 70: 1106
Day 80: 1373
Day 90: 1844
Day 100: 2208
```
After executing this process a total of 100 times, there would be 2208 black tiles facing up.

How many tiles will be black after 100 days?



In [135]:
from __future__ import annotations
from collections import Counter

def get_neighbors(p: Point):
    return set(f(p) for f in move.values())


class TileGame:
    
    def __init__(self, black_tiles: List[Point]):
        self.black_tiles = set(black_tiles)
        
    @classmethod
    def from_dict(cls, state: Dict[Point]) -> TileGame:
        return cls([point for point, color in state.items() if color == -1])
    
    def step(self):
        """
        Any black tile with zero or more than 2 black tiles immediately adjacent to it is flipped to white.
        Any white tile with exactly 2 black tiles immediately adjacent to it is flipped to black.
        """
        new_state = set()
        white_tiles = Counter()
        for tile in self.black_tiles:
            neighbors = get_neighbors(tile)
            black_neighbors = self.black_tiles & neighbors
            white_neighbors = neighbors - self.black_tiles
            if len(black_neighbors) in (1,2):
                new_state.add(tile)
            
            white_tiles.update(white_neighbors)
        #print(f"{new_state=}, {len(new_state)}")
        #print(f"{white_tiles}")
        new_state.update(k for k, v in white_tiles.items() if v == 2)
        
        self.black_tiles = new_state
            
    
    def play(self, n_turns):
        for turn in range(n_turns):
            self.step()
            print(f"After {turn=}  {self.score=}")
        print(self.score)
        
    @property
    def score(self):
        return len(self.black_tiles)

In [136]:
game = TileGame.from_dict(tiles)

In [137]:
game.score

420

In [138]:
game.play(100)

After turn=0  self.score=416
After turn=1  self.score=418
After turn=2  self.score=448
After turn=3  self.score=446
After turn=4  self.score=509
After turn=5  self.score=513
After turn=6  self.score=479
After turn=7  self.score=499
After turn=8  self.score=546
After turn=9  self.score=563
After turn=10  self.score=604
After turn=11  self.score=640
After turn=12  self.score=647
After turn=13  self.score=628
After turn=14  self.score=715
After turn=15  self.score=756
After turn=16  self.score=741
After turn=17  self.score=707
After turn=18  self.score=846
After turn=19  self.score=835
After turn=20  self.score=853
After turn=21  self.score=895
After turn=22  self.score=884
After turn=23  self.score=910
After turn=24  self.score=959
After turn=25  self.score=1015
After turn=26  self.score=967
After turn=27  self.score=1001
After turn=28  self.score=1043
After turn=29  self.score=1067
After turn=30  self.score=1115
After turn=31  self.score=1107
After turn=32  self.score=1095
After turn=33